In [1]:
import pandas as pd
import bs4
import re

from tqdm.notebook import tqdm
tqdm.pandas()

### Getting the files
We have the raw files from the database.
These are extracted in **Microsoft SQL Manager Studio 18**, by calling a `SELECT * FROM TABLE`.
Then, on the result-set right-click and press `save result as`.

In [2]:
PATH_REQUESTS = 'data/database/Requests.csv'

df = pd.read_csv(PATH_REQUESTS,
                 encoding='UTF-8',
                 delimiter=';',
                 quotechar='"',
                 dtype=str,
                 usecols=['id', 'timeConsumption'])

df = df.fillna('')

In [3]:
df = df[df['timeConsumption'] != '']
df['timeConsumption_reduced'] = df.apply(
    lambda x: float(x['timeConsumption']) if float(x['timeConsumption']) < 50.0 else 50.0, axis=1)

df['label_time'] = df['timeConsumption_reduced']
df['label_bins'] = pd.cut(df['label_time'], bins=[0.0, 2.0, 5.0, 10.0, 25.0, 50.0], labels=[0, 1, 2, 3, 4])
df[['id', 'label_bins', 'label_time']].to_csv('data/label_time.csv', index=False)

In [4]:
# df_out = pd.read_csv('data/label_time.csv')
# df_out.describe()
# df_out['label_bins'].plot(kind='hist')
# df['label_time'].plot(kind='hist')

In [5]:
import numpy as np
import pandas as pd

df_user = pd.read_csv('data/label_users_top_100.csv')
df_time = pd.read_csv('data/label_time.csv')

print(len(df_user))

df = pd.merge(df_user, df_time, on='id', how='left')
df = df.fillna('')
df = df[df['label_bins'] != '']

df[['id', 'label_bins']].to_csv('data/label_bins.csv', index=False)

df['tmp'] = (df.label_encoded * 5) + df.label_bins
df['label_time_encoded'] = df.apply(lambda x: int(x['tmp']), axis=1)

df[['id', 'label_closed', 'label_time_encoded']].to_csv('data/label_time_encoded.csv', index=False)

257899
